In [2]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame 

import matplotlib.pyplot as plt 
import seaborn as sns
import itertools
%matplotlib inline
from matplotlib.pyplot import figure
import matplotlib as mpl
import matplotlib.font_manager as fm
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'Malgun Gothic' 
plt.rcParams["font.size"] = 10
%matplotlib inline 


from datetime import datetime, timedelta 
from statsmodels.tsa.arima_model import ARIMA 

from sklearn.preprocessing import StandardScaler
import os

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Weather 추가변수
- 나중에 모델 돌릴때마다 df에 추가하는 것

In [4]:
# hourly WEATHER data
# w_hr = pd.read_csv("/kaggle/input/weather/hourly.csv",encoding='euc-kr', sep=',')
w_hr = pd.read_csv("data/weather/hourly.csv", sep=',', encoding='euc-kr')
w_hr2 = w_hr.loc[:,['Time', 'Temp','Humidity', 'WindSpeed']]
w_hr2['Humidity']=w_hr2.loc[:,'Humidity'].fillna(0)
w_hr2.head(2)

,Time,Temp,Humidity,WindSpeed
0,2016.7.20 0:00,25.7,0.0,1.3
1,2016.7.20 1:00,25.6,0.0,1.4


In [5]:
w_hr2['Time'] = pd.to_datetime(w_hr2.Time)
w_hr2 = w_hr2.set_index('Time')
w_hr2.head(2)

,Temp,Humidity,WindSpeed
Time,,,
2016-07-20 00:00:00,25.7,0.0,1.3
2016-07-20 01:00:00,25.6,0.0,1.4


## 파생 변수

In [6]:
# test = pd.read_csv("/kaggle/input/dataset/TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')
test = pd.read_csv("data/TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')
train = pd.read_csv("data/TRAIN_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')

In [7]:
test.index = pd.to_datetime(test.index)
train.index = pd.to_datetime(train.index)

In [8]:
def dateTimeVar(df):
    df1 = pd.DataFrame(df.index)
    df1.index = pd.to_datetime(df1.iloc[:,0])
    df1 = df1.drop(columns=df1.columns)
    # datetime을 사용한 파생변수
    # 연도, 월, 일, 요일, 시간
    df1['Mth']= df1.index.month
    df1['Date']=df1.index.day
    df1['Day']=df1.index.weekday
    df1['hour']=df1.index.hour
    
    # Season 1: 봄 ~ 4: 겨울 
    df1['season'] = df1.Mth.apply(lambda x: 4 if x in [12,1,2] else 
                1 if x in [3,4,5] else 2 if x in [6,7,8]
               else 3)
    # dayClass 1: 월, 2: 화~금, 3: 토, 일 
    # 참조 from 실시간 수요예측 기법 개발 및 온라인 수요예측 방안 연구_KPX 한국전력거래소)
    df1['dayClass'] = df1.Day.apply(lambda x: 1 if x ==0 else 
                2 if x in [1,2,3,4] else 3)
    return df1

In [9]:
TE_time_vars = dateTimeVar(test)

In [10]:
TE_time_vars.head(5)

,Mth,Date,Day,hour,season,dayClass
Time,,,,,,
2017-08-29 10:00:00,8,29,1,10,2,2
2017-08-29 11:00:00,8,29,1,11,2,2
2017-08-29 12:00:00,8,29,1,12,2,2
2017-08-29 13:00:00,8,29,1,13,2,2
2017-08-29 14:00:00,8,29,1,14,2,2


## Make DUMMIES

In [11]:
test.head(2)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.29667
2017-08-29 11:00:00,1.458903,0.398,0.293,0.358,0.350,0.232,0.253,0.260,0.168,0.084,...,0.163,0.086,0.288,0.342010,1.150483,0.196,0.343,1.109,0.225,0.29667


In [12]:
TE_time_vars.head(2)

,Mth,Date,Day,hour,season,dayClass
Time,,,,,,
2017-08-29 10:00:00,8,29,1,10,2,2
2017-08-29 11:00:00,8,29,1,11,2,2


In [13]:
def dummies(df, dum_lst):
    df1 = df.copy()
    # dum_lst에 들어있는 각각의 변수마다 
    # tmp = get_dummies, 해당 original 변수는 drop
    # tmp와 변수가 drop된 df를 join 후 리턴
    for v in dum_lst:
        tmp = pd.get_dummies(df1[v], prefix=v)
        df1 = df1.drop(columns=v).join(tmp)
    return df1

In [14]:
TE_timeV_dum = dummies(TE_time_vars, ['season', 'dayClass'])

In [15]:
TE_timeV_dum_wthr= pd.concat([TE_timeV_dum, w_hr2], axis=1,join='inner')

In [16]:
TE_timeV_dum_wthr.head(10)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed
Time,,,,,,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0,19.3,0.0,5.0
2017-08-29 11:00:00,8,29,1,11,0,1,0,0,0,1,0,19.7,0.0,4.7
2017-08-29 12:00:00,8,29,1,12,0,1,0,0,0,1,0,19.4,0.0,4.8
2017-08-29 13:00:00,8,29,1,13,0,1,0,0,0,1,0,19.8,0.0,4.5
2017-08-29 14:00:00,8,29,1,14,0,1,0,0,0,1,0,20.5,0.0,3.4
2017-08-29 15:00:00,8,29,1,15,0,1,0,0,0,1,0,20.6,0.0,4.0
2017-08-29 16:00:00,8,29,1,16,0,1,0,0,0,1,0,22.2,0.0,5.2
2017-08-29 17:00:00,8,29,1,17,0,1,0,0,0,1,0,21.7,0.0,5.5
2017-08-29 18:00:00,8,29,1,18,0,1,0,0,0,1,0,21.2,0.0,5.3


# Modeling

* test : TEST셋의 순수한 meterIDs Time series (NA값 채워짐) 
* TE_timeV_dum: TEST셋의 시간파생변수 + dummy variables 

In [17]:
test.head(1)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.29667


In [18]:
TE_timeV_dum_wthr.head(1)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed
Time,,,,,,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0,19.3,0.0,5.0


<br><br><br>


## Random sample 된 30개만 돌리기 (06.20~30 예측)

In [19]:
rs = pd.read_csv("data/train_all.csv", index_col=0, sep=',')
ids = rs.columns

In [20]:
test = pd.read_csv("data/TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')
train = pd.read_csv("data/TRAIN_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')

In [21]:
te = ['X41','X50','X30', 'X37', 'X18','X231', 'X116', 'X149', 'X371', 'X218']
tr = ['X40', 'X84' ,'X77', 'X93', 'X89' ,'X15', 'X23', 'X60', 'X27', 'X6','X610' ,'X1362' ,'X476', 'X237' ,'X1032', 'X1287', 'X428', 'X768', 'X989' ,'X616']

In [22]:
test.loc[:,te].head(2)

,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,
2017-08-29 10:00:00,2.167430,4.171678,3.498120,3.816439,2.518855,0.216611,0.602,0.255,0.214,0.332322
2017-08-29 11:00:00,2.159356,4.122120,3.461189,3.801315,2.519404,0.216574,0.602,0.428,0.256,0.332338


In [23]:
train.loc[:,tr]

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,X610,X1362,X476,X237,X1032,X1287,X428,X768,X989,X616
Time,,,,,,,,,,,,,,,,,,,,
2018-02-15 00:00:00,10.592437,0.164,0.561,0.340,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,0.066000,0.132000,0.091000,0.318000,0.277000,0.096000,0.020000,1.339000,0.113000,0.351000
2018-02-15 01:00:00,10.592437,0.161,0.486,0.344,1.394,2.284564,2.691859,2.094527,4.322733,4.749095,0.126000,0.278000,0.244000,0.202000,0.474000,0.338000,0.206000,0.599000,0.178000,1.495000
2018-02-15 02:00:00,10.592437,0.123,0.569,0.350,1.407,2.284564,2.691859,2.094527,4.322733,4.749095,0.227000,0.282000,0.252000,0.190000,0.302000,0.222000,0.134000,0.515000,0.223000,0.859000
2018-02-15 03:00:00,10.592437,0.115,0.590,0.333,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,0.125000,0.286000,0.254000,0.163000,0.349000,0.282000,0.223000,0.458000,0.186000,0.492000
2018-02-15 04:00:00,10.592437,0.130,0.519,0.345,1.401,2.284564,2.691859,2.094527,4.322733,4.749095,0.135000,0.266000,0.251000,0.196000,0.305000,0.257000,0.266000,0.415000,0.212000,0.471000
2018-02-15 05:00:00,10.592437,0.113,0.517,0.347,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,0.087000,0.292000,0.241000,0.198000,0.284000,0.258000,0.083000,0.475000,0.207000,0.374000
2018-02-15 06:00:00,10.592437,0.116,0.501,0.342,1.409,2.284564,2.691859,2.094527,4.322733,4.749095,0.104000,0.288000,0.423000,0.183000,0.313000,0.215000,0.089000,0.564000,0.290000,0.315000
2018-02-15 07:00:00,10.592437,0.114,0.556,0.337,1.395,2.284564,2.691859,2.094527,4.322733,4.749095,0.568000,0.615000,0.665000,0.150000,0.500000,0.218000,0.101000,0.479000,0.320000,0.355000
2018-02-15 08:00:00,10.592437,0.131,0.539,0.345,1.382,2.284564,2.691859,2.094527,4.322733,4.749095,0.677000,0.402000,0.916000,0.193000,0.352000,0.253000,0.335000,0.563000,0.828000,1.154000


In [24]:
test.loc['2018-02-15 00:00:00':,te]

,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,
2018-02-15 00:00:00,2.371049,6.545218,4.803182,4.499756,2.495460,0.327000,0.199,0.261000,0.142000,0.361000
2018-02-15 01:00:00,2.334381,5.984280,4.529291,4.341709,2.500658,0.202000,0.168,0.391000,0.319000,0.241000
2018-02-15 02:00:00,2.302862,5.544196,4.305131,4.216720,2.504838,0.213000,0.123,0.248000,0.306000,0.284000
2018-02-15 03:00:00,2.275769,5.198929,4.121671,4.117873,2.508198,0.154000,0.113,0.215000,0.280000,0.295000
2018-02-15 04:00:00,2.252481,4.928049,3.971522,4.039701,2.510895,0.141000,0.167,0.211000,0.294000,0.267000
2018-02-15 05:00:00,2.232463,4.715530,3.848635,3.977879,2.513058,0.154000,0.182,0.219000,0.433000,0.347000
2018-02-15 06:00:00,2.215256,4.548798,3.748061,3.928988,2.514790,0.150000,0.226,0.214000,0.300000,0.706000
2018-02-15 07:00:00,2.200466,4.417989,3.665748,3.890323,2.516175,0.186000,0.115,0.266000,0.418000,0.514000
2018-02-15 08:00:00,2.187752,4.315363,3.598380,3.859746,2.517280,0.134000,0.115,0.404000,0.400000,0.675000


In [27]:
data = pd.concat([train.loc[:, tr], test.loc['2018-02-15 00:00:00':,te]], axis=1 )
data.index = pd.to_datetime(data.index)

c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [28]:
hourly = pd.read_csv("data/TRAIN_DHWonDacon_24시간예측.csv",index_col=0)
hourly.head(2)

,X692,X1272,X553,X1299,X598,X1003,X1010,X1216,X1047,X381,...,X1164,X656,X1287,X324,X767,X1271,X1288,X58,X1033,X1163
1,0.374392,0.444582,0.577257,0.239870,0.358361,0.523760,0.634239,0.356952,0.265332,0.234563,...,0.440065,0.421038,0.329439,0.351530,0.578465,0.649527,0.640117,7.835084,0.331562,0.371070
2,0.366835,0.422568,0.556617,0.211252,0.310659,0.447532,0.482711,0.331569,0.265641,0.213087,...,0.416521,0.426182,0.321118,0.289726,0.486627,0.600352,0.603710,7.066954,0.338846,0.350562


In [31]:
data2 = data.copy()

In [32]:
data3 = pd.DataFrame()   # 새로운 데이터프레임 생성
n=0  # 현재 칼럼카운트를 보여주기 위한 변수

# 미터ID 리스트 c에서 1개씩 for loop
for i in data2.columns:
    if i in hourly.columns:
        n +=1   # 카운트 
        li = data2[data2[i].isnull()].index   # li: test데이터프레임의 특정 열(i)에서 Null인 셀 index추출 
        df = pd.DataFrame(data2[i])    # df: test의 i열을 별개 데이터프레임으로 생성
        print("%d. current column: %s, NaN count: %d" %(n, i, len(li)))   
        for t in li:    #  li(Null 인덱스)를 사용해 for loop
            df.loc[t] = hourly.loc[:,i].iloc[t.hour]  # null값에 t.hour 평균 대입
        data3 = pd.concat([data3,df], axis=1)   # 평균 대치가 끝난 df를 test2에 concat해준다. 

1. current column: X40, NaN count: 116
2. current column: X84, NaN count: 116
3. current column: X77, NaN count: 116
4. current column: X93, NaN count: 116
5. current column: X89, NaN count: 116
6. current column: X15, NaN count: 116
7. current column: X23, NaN count: 116
8. current column: X60, NaN count: 116
9. current column: X27, NaN count: 116
10. current column: X6, NaN count: 116
11. current column: X610, NaN count: 116
12. current column: X1362, NaN count: 116
13. current column: X476, NaN count: 116
14. current column: X237, NaN count: 116
15. current column: X1032, NaN count: 116
16. current column: X1287, NaN count: 116
17. current column: X428, NaN count: 116
18. current column: X768, NaN count: 116
19. current column: X989, NaN count: 116
20. current column: X616, NaN count: 116


In [33]:
train_data = pd.concat([data3, test.loc['2018-02-15 00:00:00':,te]], axis=1 )
train_data.index = pd.to_datetime(train_data.index)
train_data.head(3)

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,...,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
2018-02-15 00:00:00,10.592437,0.164,0.561,0.340,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.371049,6.545218,4.803182,4.499756,2.495460,0.327,0.199,0.261,0.142,0.361
2018-02-15 01:00:00,10.592437,0.161,0.486,0.344,1.394,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.334381,5.984280,4.529291,4.341709,2.500658,0.202,0.168,0.391,0.319,0.241
2018-02-15 02:00:00,10.592437,0.123,0.569,0.350,1.407,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.302862,5.544196,4.305131,4.216720,2.504838,0.213,0.123,0.248,0.306,0.284


In [34]:
train_data.isnull().sum()

X40      0
X84      0
X77      0
X93      0
X89      0
X15      0
X23      0
X60      0
X27      0
X6       0
X610     0
X1362    0
X476     0
X237     0
X1032    0
X1287    0
X428     0
X768     0
X989     0
X616     0
X41      0
X50      0
X30      0
X37      0
X18      0
X231     0
X116     0
X149     0
X371     0
X218     0
dtype: int64

In [35]:
train = train_data.loc[:'2018-06-20 23:00:00', :]
train.head()

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,...,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
2018-02-15 00:00:00,10.592437,0.164,0.561,0.340,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.371049,6.545218,4.803182,4.499756,2.495460,0.327,0.199,0.261,0.142,0.361
2018-02-15 01:00:00,10.592437,0.161,0.486,0.344,1.394,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.334381,5.984280,4.529291,4.341709,2.500658,0.202,0.168,0.391,0.319,0.241
2018-02-15 02:00:00,10.592437,0.123,0.569,0.350,1.407,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.302862,5.544196,4.305131,4.216720,2.504838,0.213,0.123,0.248,0.306,0.284
2018-02-15 03:00:00,10.592437,0.115,0.590,0.333,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.275769,5.198929,4.121671,4.117873,2.508198,0.154,0.113,0.215,0.280,0.295
2018-02-15 04:00:00,10.592437,0.130,0.519,0.345,1.401,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.252481,4.928049,3.971522,4.039701,2.510895,0.141,0.167,0.211,0.294,0.267


In [36]:
train.isnull().sum()

X40      0
X84      0
X77      0
X93      0
X89      0
X15      0
X23      0
X60      0
X27      0
X6       0
X610     0
X1362    0
X476     0
X237     0
X1032    0
X1287    0
X428     0
X768     0
X989     0
X616     0
X41      0
X50      0
X30      0
X37      0
X18      0
X231     0
X116     0
X149     0
X371     0
X218     0
dtype: int64

In [37]:
features = TE_timeV_dum_wthr.loc['2018-02-15 00:00:00':'2018-06-20 23:00:00', :]
features.head()

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed
Time,,,,,,,,,,,,,,
2018-02-15 00:00:00,2,15,3,0,0,0,0,1,0,1,0,-1.0,0.0,5.0
2018-02-15 01:00:00,2,15,3,1,0,0,0,1,0,1,0,-1.2,0.0,3.4
2018-02-15 02:00:00,2,15,3,2,0,0,0,1,0,1,0,-1.2,0.0,3.9
2018-02-15 03:00:00,2,15,3,3,0,0,0,1,0,1,0,-1.5,0.0,3.5
2018-02-15 04:00:00,2,15,3,4,0,0,0,1,0,1,0,-1.9,0.0,5.3


In [38]:
test_real = train_data.loc['2018-06-21 00:00:00':, :]
test_real.head()

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,...,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
2018-06-21 00:00:00,10.500,0.278,0.102,0.363,1.434,0.334,0.314,5.918,1.828,7.190,...,2.752,5.121,0.668,5.521,2.575,0.145,0.288,0.415,0.329,0.157
2018-06-21 01:00:00,9.905,0.275,0.098,0.360,1.416,1.332,0.298,4.968,1.032,6.841,...,2.723,5.077,0.389,5.110,2.591,0.143,0.172,0.338,0.325,0.191
2018-06-21 02:00:00,9.261,0.233,0.086,0.347,1.398,1.346,0.154,0.908,0.992,6.648,...,2.749,4.136,0.417,3.060,2.563,0.133,0.160,0.341,0.306,0.162
2018-06-21 03:00:00,9.648,0.191,0.086,0.354,1.383,1.352,0.417,1.079,0.961,6.520,...,2.731,3.142,0.412,2.287,2.568,0.132,0.204,0.382,0.297,0.150
2018-06-21 04:00:00,3.945,0.191,0.086,0.388,1.426,1.337,0.216,1.235,0.948,6.645,...,2.837,3.148,0.375,2.717,2.572,0.144,0.189,0.401,0.287,0.221


In [39]:
X_test = TE_timeV_dum_wthr.loc['2018-06-21 00:00:00':, :]
X_test.head()

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed
Time,,,,,,,,,,,,,,
2018-06-21 00:00:00,6,21,3,0,0,1,0,0,0,1,0,19.2,0.0,2.4
2018-06-21 01:00:00,6,21,3,1,0,1,0,0,0,1,0,19.0,0.0,2.5
2018-06-21 02:00:00,6,21,3,2,0,1,0,0,0,1,0,19.1,0.0,1.4
2018-06-21 03:00:00,6,21,3,3,0,1,0,0,0,1,0,19.1,0.0,1.1
2018-06-21 04:00:00,6,21,3,4,0,1,0,0,0,1,0,19.0,0.0,1.9


In [41]:
mID = 'X40'
data = pd.concat([train.loc[:,[mID]], features], axis=1)
data = data.rename(columns={mID:'y'})
for i in [24, 168]:
    data['lag_{}'.format(i)] = data.y.shift(i)
y_train = data.dropna().y
X_train = data.dropna().drop(['y'], axis=1)

y_train

2018-02-22 00:00:00    10.592437
2018-02-22 01:00:00    10.592437
2018-02-22 02:00:00    10.592437
2018-02-22 03:00:00    10.592437
2018-02-22 04:00:00    10.592437
                         ...    
2018-06-20 17:00:00    13.591000
2018-06-20 18:00:00    11.229000
2018-06-20 21:00:00    11.256000
2018-06-20 22:00:00     9.903000
2018-06-20 23:00:00    11.391000
Name: y, Length: 2834, dtype: float64

In [42]:
data.iloc[-168:,0]

2018-06-14 00:00:00    14.083
2018-06-14 01:00:00    14.528
2018-06-14 02:00:00    12.592
2018-06-14 03:00:00    11.551
2018-06-14 04:00:00     3.210
                        ...  
2018-06-20 19:00:00     9.955
2018-06-20 20:00:00    11.330
2018-06-20 21:00:00    11.256
2018-06-20 22:00:00     9.903
2018-06-20 23:00:00    11.391
Name: y, Length: 168, dtype: float64

In [43]:
np.append(data.iloc[-168:,0].values, data.iloc[-168:-96,0].values)

array([14.083     , 14.528     , 12.592     , 11.551     ,  3.21      ,
        2.898     ,  2.672     ,  3.506     , 10.163     , 10.159     ,
       10.239     , 10.478     ,  9.951     ,  9.844     , 10.743     ,
        9.991     , 10.369     , 13.66      , 16.504     , 15.298     ,
       15.253     , 15.784     , 14.767     , 17.093     , 15.376     ,
       14.33      , 12.952     , 12.24      ,  4.035     ,  3.2       ,
        3.284     ,  3.646     , 10.671     , 11.893     , 11.533     ,
       10.836     , 10.318     , 12.284     , 14.025     , 14.962     ,
       14.589     , 15.062     , 15.112     , 14.75      , 15.338     ,
       16.535     , 16.529     , 14.18      , 17.403     , 18.255     ,
       14.229     , 11.879     ,  3.943     ,  3.257     ,  3.266     ,
        3.927     , 10.469     , 10.19      , 10.091     , 11.959     ,
       15.293     , 14.543     , 14.618     , 16.954     , 16.849     ,
       16.133     , 10.65196151, 10.63407842, 10.61986245, 10.60

In [44]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

lg_param = {"boosting_type":['gbdt', 'dart','goss'],
    "learning_rate": [0.01,0.05,0.1,0.15],
    "min_child_samples":[10, 20, 30, 50],       
    "n_estimators":[10, 50, 100],
    }

def gsLGBR_Hrly_FitPred (model, params, mID, X_train, y_train, X_test):
    grid_gb = GridSearchCV(model, params, cv=4, verbose=5)
    grid_gb.fit(X_train, y_train)
    fcst = grid_gb.predict(X_test)
    a = pd.DataFrame([mID], columns=['meter_id'])
    for i in range(240):
        a[i+1]=[fcst[i]]
    return a
   
def onlytesting(train, features, X_test):
    hrly = pd.DataFrame()
    for mID in train.columns:
        data = pd.concat([train.loc[:,[mID]], features], axis=1)
        data = data.rename(columns={mID:'y'})
        for i in [24, 168]:
            data['lag_{}'.format(i)] = data.y.shift(i)
        y_train = data.dropna().y
        X_train = data.dropna().drop(['y'], axis=1)
        
        X_test['lag_24'] = y_train.tail(24).values
        X_test['lag_168'] = np.append(data.iloc[-168:,0].values, data.iloc[-168:-96,0].values)
        
        model = LGBMRegressor(verbose=1, tree_learner='feature')
        tmp = gsLGBR_Hrly_FitPred(model, lg_param,
                        mID, X_train, y_train, X_test_H)
        hrly = pd.concat([hrly_50, tmp], axis=0)
    return hrly

ModuleNotFoundError: No module named 'lightgbm'

In [45]:
train2 = train.iloc[:,0:2]
train2.head(2)

,X40,X84
2018-02-15 00:00:00,10.592437,0.164
2018-02-15 01:00:00,10.592437,0.161


In [35]:
trial = onlytesting(train2, features, X_test)

NameError: name 'X_test_H' is not defined